In [1]:
#pip install seaborn
import warnings
warnings.filterwarnings('ignore')

from pytorch_tabular import TabularModel
from pytorch_tabular.models import GatedAdditiveTreeEnsembleConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    #ExperimentConfig,
)
from pytorch_tabular.utils import get_class_weighted_cross_entropy
#pip install pytorch_tabular[extra]
from evaluation.generalevaluator import *
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris, fetch_california_housing, load_breast_cancer
from factory import create_data_loader
import pandas as pd
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from tqdm import tqdm

from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, random_split
from sklearn.utils.class_weight import compute_class_weight

In [17]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader

# Define a dummy dataset
class DummyDataset(Dataset):
    def __init__(self):
        self.X = torch.randn((100, 3))
        self.y = torch.randint(0, 2, (100, 1))

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (self.X[idx], self.y[idx])

# Create a dummy dataloader
dataset = DummyDataset()
dataloader = DataLoader(dataset, batch_size=10)

# Convert dataloader to pandas DataFrames
X_df, y_df = pd.DataFrame(), pd.DataFrame()
for X_batch, y_batch in dataloader:
    X_df = pd.concat([X_df, pd.DataFrame(X_batch.numpy())])
    y_df = pd.concat([y_df, pd.DataFrame(y_batch.numpy())])

# Optionally, you can label the columns of X_df according to the input features
X_df.columns = ["feat1", "feat2", "feat3"]

# Print the first few rows of each DataFrame
print("X_df:")
print(X_df.head())
print("y_df:")
print(y_df.head())

X_df:
      feat1     feat2     feat3
0  0.513595 -1.130417  1.612106
1 -0.034274 -0.939915 -0.185951
2  0.014506  1.940269  0.603850
3 -0.592831 -0.333628  0.700059
4 -0.525065  0.809829  0.521985
y_df:
   0
0  0
1  0
2  0
3  1
4  0


In [6]:
data_loader = create_data_loader('breastcancer', test_size=0.2, normalize_features = "mean_std", return_extra_info = True)
X_train, X_test, y_train, y_test, extra_info = data_loader.load_data()

In [9]:
X_train.shape, X_test.shape

((455, 30), (114, 30))

In [10]:
364+256+9- (455+114)

60

In [5]:
extra_info

{'cat_col_names': [],
 'cat_col_idx': [],
 'cat_col_unique_vals': [],
 'num_col_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude',
  'pop_density'],
 'num_col_idx': [0, 1, 2, 3, 4, 5, 6, 7, 8],
 'column_ordering': [4, 0, 1, 7, 3, 2, 6, 5, 8],
 'img_rows': 3,
 'img_columns': 3,
 'num_features': 9}

In [5]:
data_loader = create_data_loader('housing', test_size=0.2, normalize_features = "mean_std", return_extra_info = True)
X_train, X_test, y_train, y_test, extra_info = data_loader.load_data()

# Assume that X_train is a pandas DataFrame containing your features
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
# Select only numeric features from X_train and X_val
X_train = X_train[numeric_features]
X_test = X_test[numeric_features]

original_columns = X_train.columns
numbers_list = [16, 31, 48, 36, 11, 19, 46, 44, 14, 12, 29, 7, 33, 5, 49, 1, 3, 42, 24, 13, 15, 40, 51, 10, 9, 34, 35, 6, 47, 17, 38, 52, 0, 41, 2, 30, 23, 50, 53, 54, 4, 25, 8, 26, 18, 43, 27, 20, 32, 22, 55, 37, 39, 21, 45, 28]

new_column_ordering = [original_columns[i] for i in numbers_list]
X_train = X_train.reindex(columns=new_column_ordering)
X_test = X_test.reindex(columns=new_column_ordering)

IndexError: index 16 is out of bounds for axis 0 with size 9

In [3]:
X_train.shape

(493, 56)

In [12]:


class SoftOrdering1DCNN:
    def __init__(
        self,
        num_features,
        num_targets,
        hidden_size,
        problem_type="binary_classification",
        **params,
    ):
        self.num_features = num_features
        self.hidden_size = hidden_size
        self.problem_type = problem_type
        self.num_targets = num_targets
        self.batch_size = 512
        self.save_path = None
        self.device = params.get(
            "device", torch.device("cuda" if torch.cuda.is_available() else "cpu")
        )
        self.model = self.build_model()
        self.transformation = None

    def build_model(self):
        model = Model(self.num_features, self.num_targets, self.hidden_size)
        return model

    def train(self, X_train, y_train, params={}, extra_info=None):
        validation_fraction = params.get("validation_fraction", 0.2)
        num_epochs = params.get("num_epochs", 3)
        batch_size = params.get("batch_size", 32)
        shuffle = params.get("shuffle", True)
        validation_split = params.get("validation_split", 0.2)
        early_stopping = params.get("early_stopping", False)
        patience = params.get("early_stopping_patience", 5)
        lr = params.get("learning_rate", 0.001)
        
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        #self.optimizer = optim.SGD(self.model.parameters(), lr=lr)
        
        self.scheduler = ReduceLROnPlateau(
            self.optimizer, mode="min", factor=0.1, patience=10, verbose=True
        )
        #self.scheduler = StepLR(self.optimizer, step_size=10, gamma=0.1)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        if self.problem_type == "binary_classification":
            num_positives = y_train.sum()
            num_negatives = len(y_train) - num_positives
            pos_weight  = torch.tensor(num_negatives / num_positives)
            self.loss_fn = nn.BCEWithLogitsLoss(pos_weight = pos_weight)
        elif self.problem_type == "multiclass_classification":
            # Convert y_train to a tensor and calculate class weights for multiclass classification
            y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).flatten()
            classes = torch.unique(y_train_tensor)  # Get the unique class labels
            class_weights = compute_class_weight(
                "balanced", classes=np.array(classes), y=y_train.values
            )
            class_weights = torch.tensor(class_weights, dtype=torch.float32).to(
                self.device
            )
            self.loss_fn = nn.CrossEntropyLoss(weight=class_weights, reduction="mean")
        elif self.problem_type == "regression":
            self.loss_fn = params.get("loss_fn", nn.MSELoss())
        else:
            raise ValueError(
                "Invalid problem_type. Supported values are 'binary', 'multiclass', and 'regression'."
            )

        X_train_tensor = torch.tensor(X_train.values, dtype=torch.float)
        y_train_tensor = torch.tensor(
            y_train.values,
            dtype=torch.float if self.problem_type == "regression" else torch.long,
        )
        dataset = TensorDataset(X_train_tensor, y_train_tensor)

        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

        num_samples = len(dataset)
        num_train_samples = int((1 - validation_split) * num_samples)
        num_val_samples = num_samples - num_train_samples

        train_dataset, val_dataset = random_split(
            dataset, [num_train_samples, num_val_samples]
        )

        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

        best_val_loss = float("inf")
        best_epoch = 0
        current_patience = 0

        self.model.to(self.device)
        self.model.train()

        with tqdm(total=num_epochs, desc="Training", unit="epoch", ncols=80) as pbar:
            for epoch in range(num_epochs):
                running_loss = 0.0
                for i, (inputs, labels) in enumerate(train_loader):
                    inputs, labels = inputs.to(self.device), labels.to(self.device)

                    if (self.problem_type == "binary_classification"):
                        outputs = torch.sigmoid(self.model(inputs)).reshape(-1)
                        labels = labels.float()
                    elif self.problem_type == "regression":
                        outputs = self.model(inputs).reshape(-1)
                        labels = labels.float()
                    elif self.problem_type == "multiclass_classification":
                        labels = (
                            labels.long()
                        )  # For multiclass classification, convert to long
                        outputs = self.model(inputs)
                    else:
                        raise ValueError(
                            "Invalid problem_type. Supported options: binary_classification, multiclass_classification"
                        )

                    self.optimizer.zero_grad()

                    loss = self.loss_fn(outputs, labels)

                    loss.backward()
                    self.optimizer.step()

                    running_loss += loss.item()

                epoch_loss = running_loss / len(train_loader)

                if early_stopping and validation_fraction > 0:
                    self.model.eval()

                    val_loss = 0.0
                    with torch.no_grad():
                        for inputs, labels in val_loader:
                            inputs, labels = inputs.to(self.device), labels.to(
                                self.device
                            )

                            if (self.problem_type == "binary_classification"):
                                outputs = torch.sigmoid(self.model(inputs)).reshape(-1)
                                labels = labels.float()
                            elif self.problem_type == "regression":
                                outputs = self.model(inputs).reshape(-1)
                                labels = labels.float()
                            elif self.problem_type == "multiclass_classification":
                                labels = (
                                    labels.long()
                                )  # For multiclass classification, convert to long
                                outputs = self.model(inputs)
                            else:
                                raise ValueError(
                                    "Invalid problem_type. Supported options: binary_classification, multiclass_classification"
                                )

                            loss = self.loss_fn(outputs, labels)
                            val_loss += loss.item() * inputs.size(0)

                    val_loss /= len(val_dataset)

                    self.scheduler.step(val_loss)

                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        best_epoch = epoch
                        current_patience = 0

                        if self.save_path is not None:
                            torch.save(
                                self.model.state_dict(), self.save_path + "_checkpt"
                            )
                    else:
                        current_patience += 1

                    print(
                        f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}"
                    )

                    if current_patience >= patience:
                        print(f"Early stopping triggered at epoch {epoch+1}")
                        break

            if self.save_path is not None:
                print(f"Best model weights saved at epoch {best_epoch+1}")
                self.model.load_state_dict(torch.load(self.save_path + "_checkpt"))

    def predict(self, X_test, batch_size=4096):
        self.model.to(self.device)
        self.model.eval()

        test_dataset = CustomDataset(
            data=X_test,
            transform=None,
        )

        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        predictions = []
        probabilities = []
        with torch.no_grad():
            for inputs in test_loader:
                inputs = inputs.to(self.device)
                outputs = self.model(inputs)

                if self.problem_type == "binary_classification":
                    probs = torch.sigmoid(outputs).cpu().numpy().reshape(-1)
                    preds = (probs >= 0.5).astype(int)
                    
        
                elif self.problem_type == "multiclass_classification":
                    _, preds = torch.max(outputs, 1)
                    preds = preds.numpy()
                elif self.problem_type == "regression":
                    preds = outputs.to("cpu").tolist()
                else:
                    raise ValueError(
                        "Invalid problem_type. Supported options: binary_classification, multiclass_classification, regression."
                    )

                predictions.extend(preds)
                probabilities.extend(probs)

        return np.array(predictions), np.array(probabilities)


class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size=4196):
        super(Model, self).__init__()
        cha_1 = 256
        cha_2 = 512
        cha_3 = 512
        self.num_targets = num_targets
        cha_1_reshape = int(hidden_size / cha_1)
        cha_po_1 = int(hidden_size / cha_1 / 2)
        cha_po_2 = int(hidden_size / cha_1 / 2 / 2) * cha_3

        self.cha_1 = cha_1
        self.cha_2 = cha_2
        self.cha_3 = cha_3
        self.cha_1_reshape = cha_1_reshape
        self.cha_po_1 = cha_po_1
        self.cha_po_2 = cha_po_2

        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.1)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))

        self.batch_norm_c1 = nn.BatchNorm1d(cha_1)
        self.dropout_c1 = nn.Dropout(0.1)
        self.conv1 = nn.utils.weight_norm(
            nn.Conv1d(cha_1, cha_2, kernel_size=5, stride=1, padding=2, bias=False),
            dim=None,
        )

        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size=cha_po_1)

        self.batch_norm_c2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2 = nn.Dropout(0.1)
        self.conv2 = nn.utils.weight_norm(
            nn.Conv1d(cha_2, cha_2, kernel_size=3, stride=1, padding=1, bias=True),
            dim=None,
        )

        self.batch_norm_c2_1 = nn.BatchNorm1d(cha_2)
        self.dropout_c2_1 = nn.Dropout(0.3)
        self.conv2_1 = nn.utils.weight_norm(
            nn.Conv1d(cha_2, cha_2, kernel_size=3, stride=1, padding=1, bias=True),
            dim=None,
        )

        self.batch_norm_c2_2 = nn.BatchNorm1d(cha_2)
        self.dropout_c2_2 = nn.Dropout(0.2)
        self.conv2_2 = nn.utils.weight_norm(
            nn.Conv1d(cha_2, cha_3, kernel_size=5, stride=1, padding=2, bias=True),
            dim=None,
        )

        self.max_po_c2 = nn.MaxPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        self.batch_norm3 = nn.BatchNorm1d(cha_po_2)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(cha_po_2, num_targets))

    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.celu(self.dense1(x), alpha=0.06)

        x = x.reshape(x.shape[0], self.cha_1, self.cha_1_reshape)

        x = self.batch_norm_c1(x)
        x = self.dropout_c1(x)
        x = F.relu(self.conv1(x))

        x = self.ave_po_c1(x)

        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = F.relu(self.conv2(x))
        x_s = x

        x = self.batch_norm_c2_1(x)
        x = self.dropout_c2_1(x)
        x = F.relu(self.conv2_1(x))

        x = self.batch_norm_c2_2(x)
        x = self.dropout_c2_2(x)
        x = F.relu(self.conv2_2(x))
        x = x * x_s

        x = self.max_po_c2(x)

        x = self.flt(x)

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)

        return x


class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data.iloc[index].values.astype(np.float32)  # Convert to float32
        if self.transform:
            x = self.transform(x)
        return x

In [13]:
y_train.unique()

array([1, 0])

In [14]:
problem_type = "binary_classification"
socnn = SoftOrdering1DCNN(num_features = len(X_train.columns), num_targets = 1 ,hidden_size= 2538, problem_type = problem_type)

In [15]:
best_params = {
    'num_epochs': 10,
    'batch_size': 512,
    'shuffle': True,
    'validation_fraction': 0.15,
    'early_stopping': True,
    'early_stopping_patience': 30,
    'learning_rate': 0.001
}

In [16]:
socnn.train(X_train, y_train, params = best_params)

Training:   0%|                                       | 0/10 [00:00<?, ?epoch/s]


RuntimeError: shape '[364, 256, 9]' is invalid for input of size 923832

In [175]:
print(socnn.predict(X_test)[0])
print(socnn.predict(X_test)[1])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0.502207   0.5021919  0.50224733 0.50222707 0.5022724  0.5022734
 0.50225407 0.5021931  0.50223786 0.50221735 0.5021982  0.50226986
 0.50199884 0.50293005 0.50224966 0.502193   0.50227964 0.502275
 0.50226754 0.5022299  0.5020732  0.5021633  0.50221413 0.50223774
 0.5022417  0.50222313 0.5022052  0.5012899  0.5022073  0.5022323
 0.5022657  0.5022755  0.5024727  0.50221765 0.5023059  0.50241566
 0.5021885  0.50231725 0.5021963  0.50222886 0.5021593  0.50224274
 0.5022479  0.50229937 0.5022711  0.5021754  0.5021557  0.50234175
 0.5023185  0.48626894 0.50229293 0.5021674  0.5022649  0.5023246
 0.50223106 0.50225395 0.5021346  0.5022304  0.5022082  0.5021818
 0.50229293 0.5021956  0.5021376  0.5021858  0.5022331  0.5086306
 0.50219244 0.502

In [176]:
y_pred = socnn.predict(X_test)[0].reshape(-1)
y_true = y_test.values
y_pred[:3], y_test[:3]

(array([1, 1, 1]),
 49     1
 581    0
 82     0
 Name: target, dtype: int64)

In [177]:
# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy:", accuracy)

# Calculate precision
precision = precision_score(y_true, y_pred)
print("Precision:", precision)

# Calculate recall
recall = recall_score(y_true, y_pred)
print("Recall:", recall)

# Calculate F1 score
f1 = f1_score(y_true, y_pred)
print("F1 score:", f1)

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_true, y_pred)
print("ROC AUC score:", roc_auc)

from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_true, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

Accuracy: 0.1774193548387097
Precision: 0.17886178861788618
Recall: 0.9565217391304348
F1 score: 0.3013698630136986
ROC AUC score: 0.4782608695652174


array([0.        , 0.95652174])

In [ ]:
X_test

# GATE Tests

In [ ]:
total_count = 891
class_count = [549, 342]
class_weights = [total_count / count for count in class_count]
class_weights

In [ ]:
mu = class_weights[0] / class_weights[1] # 0.620
weighted_loss = get_class_weighted_cross_entropy(y_train.values.ravel(), mu=mu)
print(f"mu = {mu}")

In [ ]:
# Define the data configuration
data_config = DataConfig(
    target=['target'],
    continuous_cols= [i for i in extra_info["num_col_names"] if i != "target"],
    categorical_cols=extra_info["cat_col_names"],
)
trainer_config = TrainerConfig(
    auto_lr_find = True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size   = 32,
    max_epochs   = 1000,
)

optimizer_config = OptimizerConfig()

model_config = GatedAdditiveTreeEnsembleConfig(
    task="classification",
    tree_depth  =  6,
    num_trees   = 16,
    chain_trees = True, # akin to bagging, True is akin to boosting
    gflu_stages =  2,
    metrics=['accuracy'],
    metrics_params=[dict(task="multiclass", num_classes=2)]
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

In [ ]:
help(TrainerConfig)

In [ ]:
help(GatedAdditiveTreeEnsembleConfig)

In [ ]:
assert X_train.isnull().sum().sum() == 0

In [ ]:
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

# Merge X_train and y_train  
train = pd.concat([X_train, y_train], axis=1)

# Merge X_val and y_val
validation = pd.concat([X_val, y_val], axis=1)

tabular_model.fit(
    train=train, 
    validation=validation,
    loss=weighted_loss
)

In [ ]:
df_val.head()

In [ ]:
result  = tabular_model.evaluate(df_val)
result

In [ ]:
pred_df = tabular_model.predict(df_val)
pred_df.head()
# Add PassengerId information to the predictions
pred_df_with_ids = pd.concat([df_val_id.reset_index(drop=True), pred_df.reset_index(drop=True)], axis=1)

pred_df_with_ids = pred_df_with_ids.rename(columns={'prediction': 'Survived'})
# Display the predictions with PassengerId information
pred_df_with_ids[["PassengerId", "Survived"]].head()

In [ ]:
pred_df_with_ids["Survived"].value_counts()

In [ ]:
pred_df_with_ids[["PassengerId", "Survived"]].to_csv("/Users/mdicio/Desktop/doidownloads/kaggle_datasets/test_submission_boom.csv", index = False)

# TabTransformer Tests

In [ ]:
#pip install tab-transformer-pytorch
import torch
from tab_transformer_pytorch import FTTransformer

In [ ]:

model = FTTransformer(
    categories = (5, 5, 5, 5, 5),      # tuple containing the number of unique values within each category
    num_continuous = 10,                # number of continuous values
    dim = 32,                           # dimension, paper set at 32
    dim_out = 1,                        # binary prediction, but could be anything
    depth = 6,                          # depth, paper recommended 6
    heads = 8,                          # heads, paper recommends 8
    attn_dropout = 0.1,                 # post-attention dropout
    ff_dropout = 0.1                    # feed forward dropout
)

x_categ = torch.randint(0, 5, (1, 5))     # category values, from 0 - max number of categories, in the order as passed into the constructor above
x_numer = torch.randn(1, 10)              # numerical value

pred = model(x_categ, x_numer) # (1, 1)

In [ ]:
x_categ.shape, x_numer.shape

In [ ]:
x_categ = torch.randint(0, 5, (1, 5))  
x_categ, x_numer

In [ ]:
pred

In [ ]:
x

# Tabnet Tests

In [ ]:
# Split the train data into training and validation sets
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.callbacks import EarlyStopping

In [ ]:
help(TabNetClassifier)

In [ ]:
# Define the early stopping callback
early_stopping = EarlyStopping(early_stopping_metric = 'val_loss',
is_maximize = False,
patience = 32
)

model = TabNetClassifier()

# Train the model with early stopping
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_name=['valid'],
    eval_metric=['logloss'],
    max_epochs=2,
    batch_size = 16,
    callbacks=[early_stopping],
patience = 32)            

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
# Predict the labels of the validation data
y_pred = model.predict(X_test)
# Calculate the score using the specified metric
evaluator = Evaluator(problem_type = "multiclass_classification")
evaluator.y_true = y_val
evaluator.y_pred = y_pred
score = evaluator.evaluate_metric(metric_name = "f1")